In [1]:
import os
import json
import pandas as pd
x = os.listdir("./labels")

In [2]:
import shapely.wkt
import shapely
from shapely.geometry import Polygon
from collections import defaultdict

In [3]:
x

['guatemala-volcano_00000003_post_disaster.json',
 'guatemala-volcano_00000003_pre_disaster.json',
 'guatemala-volcano_00000005_post_disaster.json',
 'guatemala-volcano_00000005_pre_disaster.json',
 'guatemala-volcano_00000009_post_disaster.json',
 'guatemala-volcano_00000009_pre_disaster.json',
 'guatemala-volcano_00000011_post_disaster.json',
 'guatemala-volcano_00000011_pre_disaster.json',
 'guatemala-volcano_00000021_post_disaster.json',
 'guatemala-volcano_00000021_pre_disaster.json',
 'hurricane-florence_00000004_post_disaster.json',
 'hurricane-florence_00000004_pre_disaster.json',
 'hurricane-florence_00000005_post_disaster.json',
 'hurricane-florence_00000005_pre_disaster.json',
 'hurricane-florence_00000007_post_disaster.json',
 'hurricane-florence_00000007_pre_disaster.json',
 'hurricane-florence_00000013_post_disaster.json',
 'hurricane-florence_00000013_pre_disaster.json',
 'hurricane-florence_00000019_post_disaster.json',
 'hurricane-florence_00000019_pre_disaster.json',


In [3]:
#I need to get the list of disaster names by splitting by _
post_disasters = []
disaster_names = []
disaster_post_dict = {}
for label in x:
    if 'post' in label:
        post_disasters.append(label)

        disaster_name = label.split('_')[0]
        if disaster_name not in disaster_names:
            disaster_names.append(disaster_name)
# now I want to create a dictionary with disaster_names as key and list of post_disasters
for i in disaster_names:
    disaster_post_dict[i]= [d for d in post_disasters if i in d]
    

In [6]:
len(disaster_post_dict['guatemala-volcano'])

5

In [19]:
with open('labels/palu-tsunami_00000170_post_disaster.json') as json_file:
    data = json.load(json_file)


In [45]:
df_combined = pd.DataFrame()
for disaster_i in disaster_names:
    feat_types = []
    sub_types = []
    polygon_areas = []
    for file_label in disaster_post_dict[disaster_i]:
        with open('labels/'+file_label) as json_file:
            data = json.load(json_file)
            for x in data['features']['xy']:
                feat_type = x['properties']['feature_type']
                sub_type = x['properties']['subtype']
                polygon_geom = shapely.wkt.loads(x['wkt'])
                polygon_area = polygon_geom.area
                feat_types.append(feat_type)
                sub_types.append(sub_type)
                polygon_areas.append(polygon_area)
    df = pd.DataFrame(list(zip(feat_types, sub_types, polygon_areas)), 
               columns =['feature', 'sub_type', 'area'])
    df['disaster'] = disaster_i
    
    df_combined = df_combined.append(df)
        


In [34]:
df_combined.groupby(['disaster', 'sub_type']).count()

feature   area
disaster            sub_type                     
guatemala-volcano   destroyed            1      1
                    minor-damage         3      3
                    no-damage           26     26
                    un-classified        2      2
hurricane-florence  destroyed           10     10
                    major-damage       381    381
                    minor-damage        50     50
                    no-damage         1676   1676
                    un-classified      151    151
hurricane-harvey    destroyed          196    196
                    major-damage      2374   2374
                    minor-damage       871    871
                    no-damage         4168   4168
                    un-classified      106    106
hurricane-matthew   destroyed          679    679
                    major-damage       336    336
                    minor-damage      2339   2339
                    no-damage          617    617
                    un-classified      218    218
hurricane-michael   destroyed          183    183
                    major-damage       409    409
                    minor-damage      1363   1363
                    no-damage         3647   3647
                    un-classified       55     55
mexico-earthquake   destroyed            1      1
                    major-damage        31     31
                    minor-damage        84     84
                    no-damage        11284  11284
                    un-classified       11     11
midwest-flooding    destroyed           48     48
                    major-damage        25     25
                    minor-damage        41     41
                    no-damage         2277   2277
                    un-classified      141    141
palu-tsunami        destroyed         1237   1237
                    major-damage       250    250
                    no-damage        10927  10927
                    un-classified      146    146
santa-rosa-wildfire destroyed          964    964
                    major-damage         9      9
                    minor-damage        19     19
                    no-damage         3222   3222
                    un-classified       12     12
socal-fire          destroyed          456    456
                    major-damage        35     35
                    minor-damage        28     28
                    no-damage         3583   3583
                    un-classified      170    170

In [46]:
area_groupby = df_combined.groupby(['disaster', 'sub_type']).agg({'area': 'sum'})
# Change: groupby state_office and divide by sum
area_pcts = area_groupby.groupby(level=0).apply(lambda x:
                                                 100 * x / float(x.sum()))
area_pcts

area
disaster            sub_type                
guatemala-volcano   destroyed       0.841947
                    minor-damage   10.671381
                    no-damage      87.794650
                    un-classified   0.692022
hurricane-florence  destroyed       0.489183
                    major-damage   10.609269
                    minor-damage    3.814182
                    no-damage      76.216573
                    un-classified   8.870793
hurricane-harvey    destroyed       1.422283
                    major-damage   29.176408
                    minor-damage   10.693735
                    no-damage      56.724913
                    un-classified   1.982661
hurricane-matthew   destroyed       9.651970
                    major-damage   10.403393
                    minor-damage   65.786009
                    no-damage      11.538140
                    un-classified   2.620487
hurricane-michael   destroyed       1.860476
                    major-damage   13.397066
                    minor-damage   25.293376
                    no-damage      59.158193
                    un-classified   0.290889
mexico-earthquake   destroyed       0.015589
                    major-damage    0.439024
                    minor-damage    0.747187
                    no-damage      98.715902
                    un-classified   0.082298
midwest-flooding    destroyed       2.631464
                    major-damage    1.014414
                    minor-damage    2.691052
                    no-damage      91.644907
                    un-classified   2.018163
palu-tsunami        destroyed       8.575740
                    major-damage    2.738257
                    no-damage      88.135506
                    un-classified   0.550497
santa-rosa-wildfire destroyed      24.394012
                    major-damage    0.141565
                    minor-damage    0.616360
                    no-damage      74.817053
                    un-classified   0.031010
socal-fire          destroyed       7.254594
                    major-damage    0.579804
                    minor-damage    0.493632
                    no-damage      90.473770
                    un-classified   1.198200

In [37]:
df_midwest = pd.read_csv('midwest-flooding.csv')
df_midwest.groupby(['disaster', 'sub_type']).count()

Unnamed: 0  Unnamed: 0.1  feature  area
disaster         sub_type                                             
midwest-flooding destroyed            905           905      905   905
                 major-damage         133           133      133   133
                 minor-damage        3785          3785     3785  3785
                 no-damage           2367          2367     2367  2367

In [38]:
df_midwest_grp_by = df_midwest.groupby(['disaster', 'sub_type']).agg({'area': 'sum'})
# Change: groupby state_office and divide by sum
area_pcts = df_midwest_grp_by.groupby(level=0).apply(lambda x:
                                                 100 * x / float(x.sum()))
area_pcts

area
disaster         sub_type               
midwest-flooding destroyed      1.111287
                 major-damage   2.010837
                 minor-damage  52.531206
                 no-damage     44.346670

In [40]:
df_guat = pd.read_csv('test_df_guatemala-volcano0.csv')
df_guat.groupby(['disaster', 'sub_type']).count()


Unnamed: 0  feature  area
disaster          sub_type                               
guatemala-volcano destroyed             65       65    65
                  major-damage           6        6     6
                  minor-damage         109      109   109
                  no-damage             99       99    99

In [41]:
df_guat_grp_by = df_guat.groupby(['disaster', 'sub_type']).agg({'area': 'sum'})
# Change: groupby state_office and divide by sum
area_pcts = df_guat_grp_by.groupby(level=0).apply(lambda x:
                                                 100 * x / float(x.sum()))
area_pcts

area
disaster          sub_type               
guatemala-volcano destroyed      0.251300
                  major-damage   0.318737
                  minor-damage  81.596407
                  no-damage     17.833555

In [47]:
area_groupby.to_csv('truth.csv')